<a href="https://colab.research.google.com/github/alicetemah/alicetemah.github.io/blob/main/alice_standardModel_26_04_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall datasets

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.4 MB/s eta 0:00:00


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.7 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from datasets import load_dataset, concatenate_datasets, Dataset
from transformers import ResNetForImageClassification

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device, ' will be used.')

# transform input
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
#     ]),
#     'test': transforms.Compose([
#         transforms.Resize(224),
#         transforms.ToTensor(),
#         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
#     ])
# }







# datasets
fake = load_dataset('poloclub/diffusiondb', '2m_random_10k', split='train', data_dir='./')
real = load_dataset('frgfm/imagenette', '320px', split='train+validation', data_dir='./')

fake = fake.remove_columns(['prompt', 'seed', 'step', 'cfg', 'sampler', 'width', 'height', 'user_name', 'timestamp', 'image_nsfw', 'prompt_nsfw'])
real = real.remove_columns('label')

# add label column
fake = fake.map(lambda x: {'image': x['image'], 'label':1})
real = real.map(lambda x: {'image': x['image'], 'label':0})

print(fake)
print(real)

# split fake dataset into train, validation and test sets
fake_train_testvalid = fake.train_test_split(test_size=0.2)
fake_test_valid = fake_train_testvalid['test'].train_test_split(test_size=0.5)

# split real dataset into train, validation and test sets
real_train_testvalid = real.train_test_split(test_size=0.2)
real_test_valid= real_train_testvalid['test'].train_test_split(test_size=0.5)

# combine fake and real datasets into single dataset for each split
train_dataset = concatenate_datasets([fake_train_testvalid['train'], real_train_testvalid['train']]).shuffle(seed=42)
val_dataset = concatenate_datasets([fake_test_valid['train'], real_test_valid['train']]).shuffle(seed=42)
test_dataset = concatenate_datasets([fake_test_valid['test'], real_test_valid['test']]).shuffle(seed=42)




cpu  will be used.


Generating train split: 0 examples [00:00, ? examples/s]

Dataset diffusiondb downloaded and prepared to /root/.cache/huggingface/datasets/poloclub___diffusiondb/2m_random_10k-data_dir=.%2F/0.9.1/547894e3a57aa647ead68c9faf148324098f47f2bc1ab6705d670721de9d89d1. Subsequent calls will reuse this data.


Generating train split:   0%|          | 0/9469 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3925 [00:00<?, ? examples/s]

Dataset imagenette downloaded and prepared to /root/.cache/huggingface/datasets/frgfm___imagenette/320px-data_dir=.%2F/1.0.0/38929285b8abcae5c1305418e9d8fea5dd6b189bbbd22caba5f5537c7fa0f01f. Subsequent calls will reuse this data.


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/13394 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'label'],
    num_rows: 10000
})
Dataset({
    features: ['image', 'label'],
    num_rows: 13394
})


In [ ]:
print(train_dataset)
print(val_dataset)
print(test_dataset)

Dataset({
    features: ['image', 'label'],
    num_rows: 18715
})
Dataset({
    features: ['image', 'label'],
    num_rows: 2339
})
Dataset({
    features: ['image', 'label'],
    num_rows: 2340
})


In [ ]:
batch_size = 5

In [ ]:
train_dataset = train_dataset.select(range(800))
test_dataset = test_dataset.select(range(200))

In [ ]:
train_dataset = train_dataset.shuffle(seed=50)
test_dataset = test_dataset.shuffle(seed=60)

In [ ]:
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0, 0, 0), (0.5, 0.5, 0.5))
])

# Convert the dataset to PyTorch tensors
inputs = [transform(img.convert('RGB')) if img.mode == 'L' else transform(img) for img in train_dataset['image']]
labels = [label for label in train_dataset['label']]
inputs_tensor = torch.stack(inputs)
labels_tensor = torch.tensor(labels)

# Create a PyTorch TensorDataset
train_dataset_t= TensorDataset(inputs_tensor, labels_tensor)

# Create a PyTorch DataLoader
trainloader = DataLoader(train_dataset_t, batch_size=batch_size, shuffle=True)




In [ ]:
# Define the image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0, 0, 0), (0.5, 0.5, 0.5))
])

# Convert the dataset to PyTorch tensors
inputs = [transform(img.convert('RGB')) if img.mode == 'L' else transform(img) for img in test_dataset['image']]
labels = [label for label in test_dataset['label']]
inputs_tensor = torch.stack(inputs)
labels_tensor = torch.tensor(labels)

# Create a PyTorch TensorDataset
test_dataset_t = TensorDataset(inputs_tensor, labels_tensor)

# Create a PyTorch DataLoader
testloader = DataLoader(test_dataset_t, batch_size=batch_size, shuffle=True)

In [ ]:
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 187MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
model.fc  = nn.Sequential(    
    nn.Linear(in_features=2048, out_features=1, bias=True),   
    nn.Sigmoid() 
)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
from torch import optim
criterion = nn.BCELoss()
# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
epochs = 1
running_loss = 0
print_every = 10
train_val = []
test_val = []
for epoch in range(2):  # loop over the dataset multiple times

    
    for i, data in enumerate(trainloader, 0):
        
        # get the inputs
        inputs, labels = data
        #print(labels)

        # zero the parameter gradients
        optimizer.zero_grad()
        labels = labels.float()
        labels =labels.unsqueeze(-1)
        # forward + backward + optimize
        outputs = model.forward(inputs)        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5 == 0:    # print every x mini-batches
            train_val.append(round(running_loss / 5,3))
            print('[%d, %5d] train loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5*5))
            running_loss = 0.0
            
        if i % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    labels = labels.float()
                    labels =labels.unsqueeze(-1)
                    outputs = model.forward(inputs)
                    batch_loss = criterion(outputs, labels)
                    
                    test_loss += batch_loss.item()
            test_val.append(round(test_loss/len(testloader),3))
            print(f"Epoch {epoch+1}/{epochs}.. "                  
                  f"Test loss: {test_loss/len(testloader):.3f}.. ")
                  
            running_loss = 0
            model.train()

print('Finished Training')

[1,     1] train loss: 0.576
Epoch 1/1.. Test loss: 0.437.. 
[1,     6] train loss: 1.918
[1,    11] train loss: 4.658


KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# plot the data
plt.plot(range(len(train_val)), train_val)

# show the plot
plt.show()

In [ ]:

plt.plot(range(len(test_loss)), test_loss)


plt.show()